# Matemática Discreta A1 - Projeto Euler

Decidi aprender Julia para fazer o trabalho de Matemática Discreta. Essa linguagem é a queridinha do pessoal de otimização.

In [1]:
println("Hello world")

Hello world


Eu assisti um vídeo na internet chamado [The quickest introduction to Julia for pythonistas](https://www.youtube.com/watch?v=EJ7PLSrfj9M). Acaso esteja lendo este texto significa que eu não desisti no meio e fiz não fiz o trabalho em Python.

In [2]:
Fib(n) = if (n == 1 || n==0) 1 else Fib(n-1) + Fib(n-2) end

Fib (generic function with 1 method)

In [3]:
@time Φ = [if j==1 i else Fib(i) end for i in 1:15, j in 1:2]
Φ

  0.128076 seconds (209.06 k allocations: 10.213 MiB)


15×2 Array{Int64,2}:
  1    1
  2    2
  3    3
  4    5
  5    8
  6   13
  7   21
  8   34
  9   55
 10   89
 11  144
 12  233
 13  377
 14  610
 15  987

# [Problema 1](https://projecteuler.net/problem=1) (aquecimento)

If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.

Find the sum of all the multiples of 3 or 5 below 1000.


In [4]:
Mult_Or(k,a,b) = if k % a == 0 || k % b == 0 k else 0 end

Mult_Or (generic function with 1 method)

In [5]:
sum_Helper(fun, max) = if max != 0 fun(max) + sum_Helper(fun,max-1) else 0 end
my_sum(max, a, b) = sum_Helper(((candidate) -> Mult_Or(candidate,a,b)), max-1)

my_sum (generic function with 1 method)

In [6]:
my_sum(10,3,5)

23

In [7]:
@time my_sum(1000,3,5)

  0.000023 seconds (5 allocations: 176 bytes)


233168

In [8]:
@time sum([k for k = 1:999 if k % 3 == 0 || k % 5 == 0]) #mais simples, mas menos genérico, menos rápido e menos legal

  0.108090 seconds (66.94 k allocations: 3.373 MiB)


233168

In [9]:
function my_Func2(max)
    resp = 0
    for k = 1:(max-1)
        if k % 3 == 0 || k% 5 == 0
            resp += k
        end
    end
    return resp
end

my_Func2 (generic function with 1 method)

In [10]:
@time my_Func2(1000) 

  0.020095 seconds (16.64 k allocations: 887.530 KiB)


233168

In [11]:
@time my_Func2(1000) #Aparentemente, o Julia tem um memoization rolando por debaixo dos panos

  0.000009 seconds (5 allocations: 176 bytes)


233168

# [Problema 2](https://projecteuler.net/problem=2)


Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with $1$ and $2$, the first 10 terms will be:

$1, 2, 3, 5, 8, 13, 21, 34, 55, 89\ldots$

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.


**Lema:** $F_n\ |\ 2  \Rightarrow n+1\ |\ 3$

Se $F_n$ é par, existem duas possíbilidades. Ou $F_{n-1}$ e $F_{n-2}$ são ambos pares, ou são ambos ímpares. Suponha por absurdo que ambos são pares, logo isso implicaria que $F_n$ é par para todo $n$. Mas existe $F_n$ ímpar, portanto não podem $F_{n-1}$ e $F_{n-2}$ ambos pares. No caso de serem ambos ímpares, logo nota-se um padrão. Considere a sequência $(\ldots,F_{n-3},F_{n-2},F_{n-1},F_{n},F_{n+1},F_{n+2},F_{n+3},\ldots)$. Adicionemos as informações que temos. $(\ldots,F_{n-3},F_{n-2}^i,F_{n-1}^i,F_{n}^p,F_{n+1},F_{n+2},F_{n+3},\ldots)$. Ora, conseguimos preencher o resto da sequência. $(\ldots,F_{n-3}^p,F_{n-2}^i,F_{n-1}^i,F_{n}^p,F_{n+1}^i,F_{n+2}^i,F_{n+3}^p,\ldots)$. Portanto $F_n$ é par a cada $3$ números de Fibonacci. Mas sabemos que $F_2$ é par, isso implica que $n+1|3$ para todo $n$, por conta da periodicidade das aparições.

In [12]:
function Euler2(max, this_Fib)
    flag = true
    index = 2
    sum = 0
    next_Fib = this_Fib(index)
    while flag
        sum += next_Fib
        index += 3
        next_Fib = this_Fib(index)
        flag = next_Fib <= max  
    end
    return sum
end

Euler2 (generic function with 1 method)

In [13]:
@time Euler2(4000000,Fib)

  0.125439 seconds (19.47 k allocations: 1002.483 KiB)


4613732

Estamos Fibonacci definida por recursão (escrevi no começo do ipynb sem saber que iria acabar usando apenas para verificar se eu sabia escrever uma função em Julia). Vamos tentar fazer melhor, com algoritmo $O(n)$?

Um professor de matemática discreta provou que:

$F_n = \dfrac{1}{\sqrt{5}}\left[\left(\frac{1+\sqrt{5}}{2}\right)^n-\left(\frac{1-\sqrt{5}}{2}\right)^n\right]$

In [14]:
Smart_Fib(n) = (((1+sqrt(5))/2)^(n+1) - ((1-sqrt(5))/2)^(n+1))/sqrt(5)

Smart_Fib (generic function with 1 method)

In [15]:
@time trunc(Int,Euler2(4000000,Smart_Fib))

  0.056239 seconds (57.60 k allocations: 3.066 MiB)


4613732

Como esperado, ao usar um algoritmo $O(n)$, conseguimos melhorar bastante o tempo de execução.